In [1]:
import pyarrow.parquet as pq
import pandas as pd
import threading
import os
import json
import time
import gc

In [3]:
pardata = pq.ParquetFile('./part-00001.parquet')
data = pardata.read().to_pandas()
data.shape,data.columns

((1250000, 15),
 Index(['id', 'timestamp', 'user_name', 'chinese_name', 'email', 'age',
        'income', 'gender', 'country', 'chinese_address', 'purchase_history',
        'is_active', 'registration_date', 'credit_score', 'phone_number'],
       dtype='object'))

In [4]:
data.head(5)

,id,timestamp,user_name,chinese_name,email,age,income,gender,country,chinese_address,purchase_history,is_active,registration_date,credit_score,phone_number
0,1,2025-01-25T07:15:59+00:00,gtlcuz,于宏,ykxyfhgm@163.com,96,746000.0,男,日本,福建省乌鲁木齐东风路80号6单元2277,"{""average_price"":736.66,""category"":""服装"",""items...",False,2024-11-09,819,936-365-4573
1,2,2024-10-10T23:32:13+00:00,nbcpsl,马鹏,vfiteszh@hotmail.com,29,944000.0,男,日本,西藏自治区重庆新华路61号6单元662,"{""average_price"":226.81,""category"":""书籍"",""items...",False,2024-03-04,373,924-957-8702
2,3,2025-02-13T22:44:56+00:00,gsbhu,徐雨,yduzrewc@163.com,36,123000.0,女,澳大利亚,江西省长沙青年路163号6单元407,"{""average_price"":596.0799999999999,""category"":...",False,2024-12-25,419,371-760-3106
3,4,2023-04-24T03:46:06+00:00,YAAYX,林强,zutmavfa@163.com,58,132000.0,男,美国,安徽省中山振兴路61号2单元2249,"{""average_price"":494.11,""category"":""书籍"",""items...",False,2020-09-22,569,463-338-5133
4,5,2023-07-10T07:48:22+00:00,TAJJHCND,廖强,pvyzvahj@outlook.com,57,278000.0,女,巴西,澳门特别行政区南昌西湖路118号7单元377,"{""average_price"":676.2700000000001,""category"":...",False,2021-03-23,564,214-297-9938


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250000 entries, 0 to 1249999
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   id                 1250000 non-null  int64  
 1   timestamp          1250000 non-null  object 
 2   user_name          1250000 non-null  object 
 3   chinese_name       1250000 non-null  object 
 4   email              1250000 non-null  object 
 5   age                1250000 non-null  int64  
 6   income             1250000 non-null  float64
 7   gender             1250000 non-null  object 
 8   country            1250000 non-null  object 
 9   chinese_address    1250000 non-null  object 
 10  purchase_history   1250000 non-null  object 
 11  is_active          1250000 non-null  bool   
 12  registration_date  1250000 non-null  object 
 13  credit_score       1250000 non-null  int64  
 14  phone_number       1250000 non-null  object 
dtypes: bool(1), float64(1), int64(3)

In [6]:
data['id'] = data['id'].astype('str')
data.head(5)

,id,timestamp,user_name,chinese_name,email,age,income,gender,country,chinese_address,purchase_history,is_active,registration_date,credit_score,phone_number
0,1,2025-01-25T07:15:59+00:00,gtlcuz,于宏,ykxyfhgm@163.com,96,746000.0,男,日本,福建省乌鲁木齐东风路80号6单元2277,"{""average_price"":736.66,""category"":""服装"",""items...",False,2024-11-09,819,936-365-4573
1,2,2024-10-10T23:32:13+00:00,nbcpsl,马鹏,vfiteszh@hotmail.com,29,944000.0,男,日本,西藏自治区重庆新华路61号6单元662,"{""average_price"":226.81,""category"":""书籍"",""items...",False,2024-03-04,373,924-957-8702
2,3,2025-02-13T22:44:56+00:00,gsbhu,徐雨,yduzrewc@163.com,36,123000.0,女,澳大利亚,江西省长沙青年路163号6单元407,"{""average_price"":596.0799999999999,""category"":...",False,2024-12-25,419,371-760-3106
3,4,2023-04-24T03:46:06+00:00,YAAYX,林强,zutmavfa@163.com,58,132000.0,男,美国,安徽省中山振兴路61号2单元2249,"{""average_price"":494.11,""category"":""书籍"",""items...",False,2020-09-22,569,463-338-5133
4,5,2023-07-10T07:48:22+00:00,TAJJHCND,廖强,pvyzvahj@outlook.com,57,278000.0,女,巴西,澳门特别行政区南昌西湖路118号7单元377,"{""average_price"":676.2700000000001,""category"":...",False,2021-03-23,564,214-297-9938


In [7]:
age_qcut = pd.qcut(data['age'],10)
data['age group'] = age_qcut
data

,id,timestamp,user_name,chinese_name,email,age,income,gender,country,chinese_address,purchase_history,is_active,registration_date,credit_score,phone_number,age group
0,1,2025-01-25T07:15:59+00:00,gtlcuz,于宏,ykxyfhgm@163.com,96,746000.0,男,日本,福建省乌鲁木齐东风路80号6单元2277,"{""average_price"":736.66,""category"":""服装"",""items...",False,2024-11-09,819,936-365-4573,"(92.0, 100.0]"
1,2,2024-10-10T23:32:13+00:00,nbcpsl,马鹏,vfiteszh@hotmail.com,29,944000.0,男,日本,西藏自治区重庆新华路61号6单元662,"{""average_price"":226.81,""category"":""书籍"",""items...",False,2024-03-04,373,924-957-8702,"(26.0, 34.0]"
2,3,2025-02-13T22:44:56+00:00,gsbhu,徐雨,yduzrewc@163.com,36,123000.0,女,澳大利亚,江西省长沙青年路163号6单元407,"{""average_price"":596.0799999999999,""category"":...",False,2024-12-25,419,371-760-3106,"(34.0, 43.0]"
3,4,2023-04-24T03:46:06+00:00,YAAYX,林强,zutmavfa@163.com,58,132000.0,男,美国,安徽省中山振兴路61号2单元2249,"{""average_price"":494.11,""category"":""书籍"",""items...",False,2020-09-22,569,463-338-5133,"(51.0, 59.0]"
4,5,2023-07-10T07:48:22+00:00,TAJJHCND,廖强,pvyzvahj@outlook.com,57,278000.0,女,巴西,澳门特别行政区南昌西湖路118号7单元377,"{""average_price"":676.2700000000001,""category"":...",False,2021-03-23,564,214-297-9938,"(51.0, 59.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249995,49996,2024-10-20T10:29:39+00:00,gbvbso,董飞,wyhjzjbp@qq.com,70,614000.0,男,德国,甘肃省东莞富强路182号4单元1803,"{""average_price"":291.15999999999997,""category""...",False,2024-03-26,645,468-146-3816,"(67.0, 76.0]"
1249996,49997,2024-04-20T05:29:01+00:00,ddoradqex,康思,inmtliwz@hotmail.com,29,886000.0,男,中国,上海市佛山青年路119号2单元2469,"{""average_price"":418.87,""category"":""服装"",""items...",False,2023-01-21,379,989-581-5117,"(26.0, 34.0]"
1249997,49998,2023-10-14T07:27:36+00:00,inzbe,王红,zmvqpgzu@126.com,92,546000.0,未指定,澳大利亚,陕西省北京科技路6号2单元975,"{""average_price"":583.2099999999999,""category"":...",False,2021-11-03,796,981-583-6330,"(84.0, 92.0]"
1249998,49999,2024-12-20T22:35:03+00:00,ajoudxeaz,崔雪,wcnzesgb@qq.com,51,588000.0,女,法国,宁夏回族自治区保定新华路97号7单元2113,"{""average_price"":481.23999999999995,""category""...",False,2024-08-18,520,660-899-8749,"(43.0, 51.0]"


In [8]:
item = data['purchase_history'][0]
json.loads(item)

{'average_price': 736.66,
 'category': '服装',
 'items': [{'id': 295},
  {'id': 397},
  {'id': 798},
  {'id': 906},
  {'id': 916},
  {'id': 953},
  {'id': 855},
  {'id': 201},
  {'id': 140},
  {'id': 302}]}